## Load modules and packages

In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
import random
from sklearn.pipeline import Pipeline
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingRandomSearchCV

## Load data

In [2]:
random.seed(10) 

from functions import *
# Load data - set index column, decimal point, separator
data = pd.read_csv('hw1_devsample.csv', sep=',',
                   decimal='.', index_col='SK_ID_CURR')

# print time of data being loaded - use strftime
print(f'Data loaded on:   {datetime.datetime.now().strftime(format="%Y-%m-%d %H:%M:%S")}')
data_cart = data.copy()

data_test = pd.read_csv('hw1_outofsample.csv', sep=',',decimal='.', index_col='SK_ID_CURR')

Data loaded on:   2022-05-04 15:04:22


In [3]:
print(data_cart)

            TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
SK_ID_CURR                                                       
215138         0.0         Cash loans           M            N   
325963         0.0         Cash loans           M            Y   
206923         0.0         Cash loans           F            N   
265855         0.0         Cash loans           F            N   
359795         0.0    Revolving loans           F            N   
...            ...                ...         ...          ...   
203263         0.0         Cash loans           F            N   
304373         0.0         Cash loans           M            N   
348873         0.0         Cash loans           F            Y   
296666         1.0         Cash loans           M            Y   
259380         0.0         Cash loans           F            N   

           FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
SK_ID_CURR                                                         

## Small changes

In [4]:
target = data_cart['TARGET']
data_cart = data_cart.drop(['MONTH','TIME','DAY','BASE','TARGET'], axis=1)
data_cart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80000 entries, 215138 to 259380
Columns: 191 entries, NAME_CONTRACT_TYPE to LAST_TRANSACTION_TIME_MONTHS
dtypes: float64(138), int64(37), object(16)
memory usage: 117.2+ MB


In [15]:
cols_pred = data_cart.columns
cols_pred_cat = [col for col in cols_pred if data_cart[col].dtype == 'O']
cols_pred_cat


['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_cart, target, test_size=0.2, random_state=123)

## Split data train and test and Test target Encoder 

In [14]:
target_encoder = ce.target_encoder.TargetEncoder(
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value'
)

# Test XGBOOST

In [11]:
xgb_guillaume = xgb.XGBClassifier()

In [12]:
print(xgb_guillaume)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)


# Pipeline

In [13]:
pipe = Pipeline([
    ('MeanTargetEncoding', target_encoder),
    ('model', xgb_guillaume)
])

## Halving Random Search CV

In [ ]:
param_grid = {
    'MeanTargetEncoding__verbose': [0,1,2],
    'model__max_depth': [2, 4, 6, 8]
}

## Apply pipeline to data

In [ ]:
search = HalvingRandomSearchCV(pipe, param_grid, cv=3).fit(X_train, y_train)

## Calculate AUC

# Predict on the second dataset

In [ ]:
prediction = ...

In [ ]:
data_saving = pd.DataFrame({'SK_ID_CURR' : data_test.index ,'prediction' : prediction})
data_saving.to_csv('DS2_22_HW1_CADIOU&PAIN.csv',index=False,sep=',')
data_saving.info()